In [17]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1732130710129_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1732130710129_0003,pyspark,idle,Link,Link,None,✔


In [18]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("numpy")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1732130710129_0003/container_1732130710129_0003_01_000001/tmp/spark-6f1cdf6a-47ce-444c-8d08-9ca81c6ebc75
    Can't uninstall 'python-dateutil'. No files were found to uninstall.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.15.30 requires python-dateutil<=2.8.2,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.


In [50]:
from pyspark.sql.functions import expr, col, udf, desc, collect_list
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, FloatType
from pyspark.sql import functions as F
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.mllib.evaluation import RankingMetrics
from pyspark.sql import Window
from pyspark.sql.functions import row_number

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RecommenderSystem").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df = spark.read.csv("s3://proyecto-mineria-de-datos/user-filtered.csv", header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
df = df.filter(df['rating'] != 0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)

In [29]:
df = df.orderBy("user_id", "anime_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|      21|    10|
|      0|      24|     9|
|      0|      67|     9|
|      0|      68|     6|
|      0|     121|     9|
|      0|     164|     8|
|      0|     169|     7|
|      0|     174|     4|
|      0|     199|     8|
|      0|     235|    10|
|      0|     242|    10|
|      0|     269|     9|
|      0|     356|     9|
|      0|     415|    10|
|      0|     419|     8|
|      0|     430|     9|
|      0|     431|     8|
|      0|     433|     6|
|      0|     448|     5|
|      0|     459|     9|
+-------+--------+------+
only showing top 20 rows

In [31]:
df_final = df

total_users = df.select("user_id").distinct().count()
total_animes = df.select("anime_id").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
indexer_user = StringIndexer(inputCol='user_id', outputCol='user_index').setHandleInvalid("keep")
indexer_anime = StringIndexer(inputCol='anime_id', outputCol='anime_index').setHandleInvalid("keep")

user_indexer_model = indexer_user.fit(df_final)
anime_indexer_model = indexer_anime.fit(df_final)

df_final = user_indexer_model.transform(df_final)
df_final = anime_indexer_model.transform(df_final)

df_final = df_final.withColumn('user_index', df_final['user_index'].cast('integer'))
df_final = df_final.withColumn('anime_index', df_final['anime_index'].cast('integer'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
percent_users_to_mask = 0.95
percent_animes_to_mask = 0.9

user_cutoff = int(total_users * (1 - percent_users_to_mask))
anime_cutoff = int(total_animes * (1 - percent_animes_to_mask))
print(user_cutoff, total_users)
print(anime_cutoff, total_animes)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15683 313670
1717 17172

In [34]:
train_data = df_final.filter(~((col("user_index") > user_cutoff) & (col("anime_index") > anime_cutoff)))
test_data = df_final.filter((col("user_index") > user_cutoff) & (col("anime_index") > anime_cutoff))
print(train_data.count())
print(test_data.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

53656724
8740988

In [35]:
final_als = ALS(
    userCol='user_index',
    itemCol='anime_index',
    ratingCol='rating',
    coldStartStrategy='drop',
    nonnegative=True,
    rank=20,
    maxIter=20,
    regParam=0.15
)

best_model = final_als.fit(train_data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
predictions = best_model.transform(test_data)
predictions = predictions.withColumn("prediction", expr("CASE WHEN prediction < 1 THEN 1 WHEN prediction > 10 THEN 10 ELSE prediction END"))
evaluator = RegressionEvaluator(labelCol='rating', predictionCol='prediction')

metrics = {}

metric_names = ['rmse', 'mae', 'mse', 'r2']

for metric in metric_names:
    evaluator.setMetricName(metric)
    metrics[metric] = evaluator.evaluate(predictions)

print(f"Root Mean Squared Error (RMSE): {metrics['rmse']}")
print(f"Mean Absolute Error (MAE): {metrics['mae']}")
print(f"Mean Squared Error (MSE): {metrics['mse']}")
print(f"R² (coefficient of determination): {metrics['r2']}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE): 1.3907441978574457
Mean Absolute Error (MAE): 1.0616008498457217
Mean Squared Error (MSE): 1.93416942387415
R? (coefficient of determination): 0.4282145765083143

In [51]:
def evaluate_als_recommendations(model, test_data, k=10):
    user_subset = test_data.select('user_index').distinct()
    
    recommendations = model.recommendForUserSubset(user_subset, k)
    
    actual_rankings = ( test_data
        .groupBy("user_index")
        .agg(
            F.collect_list(
                F.struct("anime_index", "rating")
            ).alias("anime_ratings")
        ) \
        .select(
            "user_index", 
            F.expr("transform(reverse(sortArray(item_ratings, lambda(a, b) -> a.rating < b.rating)), x -> x.anime_index) as true_items")
        )
    )
    
    # Prepare predicted rankings RDD
    predicted_rankings = recommendations.selectExpr(
        "user_index", "recommendations.anime_index as predicted"
    )
    
    # Join predictions with ground truth
    prediction_truth = predicted_rankings.join(
        actual_rankings, on='user_index'
    )
    
    prediction_truth_rdd = prediction_truth.rdd.map(
        lambda row: (row['predicted'], row['ground_truth'])
    )

    metrics = RankingMetrics(prediction_truth_rdd)
    
    results = {
        'MAP': metrics.meanAveragePrecision,
        'NDCG at K': metrics.ndcgAt(k),
        'Precision at K': metrics.precisionAt(k),
    }
    
    return results

metrics_results = evaluate_als_recommendations(best_model, test_data, k=10)

# Print results
for metric_name, value in metrics_results.items():
    print(f"{metric_name}: {value:.4f}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MAP: 0.0002
NDCG at K: 0.0016
Precision at K: 0.0018
/mnt/yarn/usercache/livy/appcache/application_1732130710129_0003/container_1732130710129_0003_01_000001/pyspark.zip/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.